# IMPORTAR LIBRERIAS

In [1]:
def reduce_columns_period_sum(dataframe, columns, period_days):
    if 672 % (period_days*24) == 0:
        divition_length = period_days*24
        new_dataframe = pd.DataFrame()

        for column in columns:
            for i in range(int(672/divition_length)):
                new_dataframe[f'{column}_{i}'] = np.zeros(dataframe.shape[0])
                div_start = int(i * divition_length)
                for j in range(div_start, div_start + divition_length):
                    new_dataframe[f'{column}_{i}'] = new_dataframe[f'{column}_{i}'] + dataframe[f'{column}_{j}']

        return new_dataframe

    else:
        raise ValueError('28 días debe ser divisible por el período')


def reduce_columns_period_avg(dataframe, columns, period_days):
    if 672 % (period_days * 24) == 0:
        divition_length = period_days * 24
        print(divition_length)
        new_dataframe = pd.DataFrame()

        for column in columns:
            for i in range(int(672 / divition_length)):
                new_dataframe[f'{column}_{i}'] = np.zeros(dataframe.shape[0])
                div_start = int(i * divition_length)
                for j in range(div_start, div_start + divition_length):
                    new_dataframe[f'{column}_{i}'] = new_dataframe[
                        f'{column}_{i}'] + dataframe[f'{column}_{j}'] / (672 / divition_length)

        return new_dataframe

    else:
        raise ValueError('28 días debe ser divisible por el período')

In [2]:
!pip install tensorflow

In [3]:
import pandas as pd

In [4]:
from sklearn.model_selection import train_test_split

X = pd.read_csv('../raw_data/X.csv', index_col = 0)
y = pd.read_csv('../raw_data/y.csv', index_col = 0)



In [5]:
print(X.shape)
print(y.shape)

(7500, 12115)
(7500, 5376)


In [6]:
cols_t = y.iloc[: , -672:]
X = pd.concat([X, cols_t], axis = 1)
y = y.iloc[: , :-672]

In [7]:
X.shape

(7500, 12787)

In [8]:
y.shape

(7500, 4704)

In [9]:
columns_s = [x[:-4] for x in X.columns if '671' in x]
columns_s

['ac_t_conf',
 'ac_t_red',
 'ac_mask',
 'heat_t_conf',
 'heat_t_red',
 'heat_mask',
 'ventilation_t',
 'ventilation_vol',
 'ventilation_mask',
 'occupancy',
 'pc_on_mask',
 'DNI',
 'IBEAM_H',
 'IBEAM_N',
 'IDIFF_H',
 'IGLOB_H',
 'RHUM',
 'TAMB',
 'T_INT_OFFICE']

In [10]:
import numpy as np

In [11]:
X_red = reduce_columns_period_avg(X, columns_s, 28)
X_red.head()
#promedio de todos los dias en intervalos de 24 hs 

# ejemplo --- calculo el promedio en la hora 0 de todos los dia, asi hasta 24  ESTO ES MENSUAL 
#

672


,ac_t_conf_0,ac_t_red_0,ac_mask_0,heat_t_conf_0,heat_t_red_0,heat_mask_0,ventilation_t_0,ventilation_vol_0,ventilation_mask_0,occupancy_0,pc_on_mask_0,DNI_0,IBEAM_H_0,IBEAM_N_0,IDIFF_H_0,IGLOB_H_0,RHUM_0,TAMB_0,T_INT_OFFICE_0
0,15744.0,17232.0,292.0,13824.0,12672.0,284.0,12912.0,1104.0,272.0,220.0,269.2680,132889.0,90567.0,174589.0,82903.0,173471.0,41698.0,13550.0,16246.1087
1,15696.0,17232.0,288.0,13872.0,12336.0,288.0,13056.0,1296.0,244.0,220.0,282.7828,95349.0,50885.0,128549.0,51335.0,102239.0,15518.0,6968.9,14627.2995
2,15696.0,17136.0,280.0,13872.0,12432.0,316.0,12720.0,1008.0,268.0,220.0,300.2752,92963.0,56727.0,139089.0,69075.0,125822.0,40756.0,8491.8,15620.7851
3,15744.0,17136.0,268.0,13776.0,12432.0,300.0,12624.0,960.0,280.0,220.0,363.0128,37060.0,15822.0,70655.0,41543.0,57378.0,51918.0,9891.6,15325.9626
4,15744.0,17184.0,268.0,13920.0,12480.0,292.0,12912.0,864.0,280.0,220.0,360.9336,164557.0,111618.0,192637.0,79112.0,190726.0,38735.0,14364.6,16199.8736


In [12]:
columns_c = X.columns[0:19]
columns_c

Index(['airchange_infiltration_vol_per_h', 'capacitance_kJ_perdegreK_perm3',
       'power_VCV_kW_heat', 'power_VCV_kW_clim', 'nb_occupants', 'nb_PCs',
       'facade_1_thickness_2', 'facade_1_window_area_percent',
       'facade_2_thickness_2', 'facade_2_window_area_percent',
       'facade_3_thickness_2', 'facade_3_window_area_percent',
       'facade_4_thickness_2', 'facade_4_window_area_percent',
       'roof_thickness_2', 'ground_thickness_2', 'init_day', 'init_month',
       'init_year'],
      dtype='object')

In [13]:
columns_c = X.columns[0:16]

In [14]:
X = pd.concat([X[columns_c], X_red], axis = 1)
X.head()

,airchange_infiltration_vol_per_h,capacitance_kJ_perdegreK_perm3,power_VCV_kW_heat,power_VCV_kW_clim,nb_occupants,nb_PCs,facade_1_thickness_2,facade_1_window_area_percent,facade_2_thickness_2,facade_2_window_area_percent,...,occupancy_0,pc_on_mask_0,DNI_0,IBEAM_H_0,IBEAM_N_0,IDIFF_H_0,IGLOB_H_0,RHUM_0,TAMB_0,T_INT_OFFICE_0
0,0.1,20.0,500.0,500.0,1000.0,1000.0,0.15,45.0,0.06,45.0,...,220.0,269.2680,132889.0,90567.0,174589.0,82903.0,173471.0,41698.0,13550.0,16246.1087
1,0.3,30.0,600.0,500.0,1300.0,1400.0,0.09,40.0,0.11,50.0,...,220.0,282.7828,95349.0,50885.0,128549.0,51335.0,102239.0,15518.0,6968.9,14627.2995
2,0.1,270.0,500.0,600.0,1500.0,1500.0,0.08,45.0,0.07,40.0,...,220.0,300.2752,92963.0,56727.0,139089.0,69075.0,125822.0,40756.0,8491.8,15620.7851
3,0.4,140.0,500.0,600.0,1300.0,1300.0,0.08,45.0,0.08,50.0,...,220.0,363.0128,37060.0,15822.0,70655.0,41543.0,57378.0,51918.0,9891.6,15325.9626
4,0.4,40.0,600.0,600.0,1000.0,1500.0,0.06,50.0,0.07,45.0,...,220.0,360.9336,164557.0,111618.0,192637.0,79112.0,190726.0,38735.0,14364.6,16199.8736


In [15]:
y_red = reduce_columns_period_sum(y, ['Q_AC_OFFICE', 'Q_HEAT_OFFICE', 'Q_PEOPLE', 'Q_EQP', 'Q_LIGHT', 'Q_AHU_C', 'Q_AHU_H'], 28)
y_red.head()

,Q_AC_OFFICE_0,Q_HEAT_OFFICE_0,Q_PEOPLE_0,Q_EQP_0,Q_LIGHT_0,Q_AHU_C_0,Q_AHU_H_0
0,10748.1049,0.0000,3240.0,3874.3458,2112.2700,3890.5751,1096.6969
1,1819.3670,469.0627,4212.0,5667.4181,2151.4910,40.6438,7093.4597
2,6245.3497,0.0000,4860.0,6409.7036,2027.7509,662.0432,4612.1041
3,2372.0553,0.0000,4212.0,6605.6016,2739.8060,285.1977,2852.4764
4,15234.8404,0.0000,3240.0,7581.4202,2093.6939,3937.5324,520.0846


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y_red, test_size=0.33, random_state=42)

In [17]:
from sklearn.preprocessing import StandardScaler

s_scaler = StandardScaler()

X_train_scaled = pd.DataFrame(s_scaler.fit_transform(X_train))
X_test_scaled = pd.DataFrame(s_scaler.transform(X_test))
y_train_scaled = pd.DataFrame(s_scaler.fit_transform(y_train))
y_test_scaled = pd.DataFrame(s_scaler.transform(y_test))

In [18]:
X_train_scaled.columns = X_train.columns
X_test_scaled.columns = X_test.columns
y_train_scaled.columns = y_train.columns
y_test_scaled.columns = y_test.columns

In [19]:
X_train_scaled.shape

(5025, 35)

In [20]:
X_train_scaled.columns

Index(['airchange_infiltration_vol_per_h', 'capacitance_kJ_perdegreK_perm3',
       'power_VCV_kW_heat', 'power_VCV_kW_clim', 'nb_occupants', 'nb_PCs',
       'facade_1_thickness_2', 'facade_1_window_area_percent',
       'facade_2_thickness_2', 'facade_2_window_area_percent',
       'facade_3_thickness_2', 'facade_3_window_area_percent',
       'facade_4_thickness_2', 'facade_4_window_area_percent',
       'roof_thickness_2', 'ground_thickness_2', 'ac_t_conf_0', 'ac_t_red_0',
       'ac_mask_0', 'heat_t_conf_0', 'heat_t_red_0', 'heat_mask_0',
       'ventilation_t_0', 'ventilation_vol_0', 'ventilation_mask_0',
       'occupancy_0', 'pc_on_mask_0', 'DNI_0', 'IBEAM_H_0', 'IBEAM_N_0',
       'IDIFF_H_0', 'IGLOB_H_0', 'RHUM_0', 'TAMB_0', 'T_INT_OFFICE_0'],
      dtype='object')

In [21]:
y_train_scaled.columns

Index(['Q_AC_OFFICE_0', 'Q_HEAT_OFFICE_0', 'Q_PEOPLE_0', 'Q_EQP_0',
       'Q_LIGHT_0', 'Q_AHU_C_0', 'Q_AHU_H_0'],
      dtype='object')

In [22]:
y_train_scaled.shape

(5025, 7)

# RED 

In [23]:
from tensorflow.keras import *

2021-10-19 18:57:38.703728: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-19 18:57:38.703786: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from keras import backend as K

In [40]:
def det_coeff(y_test_scaled, y_pred):
    u = K.sum(K.square(y_test_scaled - y_pred))
    v = K.sum(K.square(y_test_scaled - K.mean(y_test_scaled)))
    return K.ones_like(v) - (u / v)

In [41]:
def initialize_model():
    
    model = Sequential()
    model.add(layers.Dense(1500, activation='relu',input_dim=35))    
       
    model.add(layers.Dense(250, activation='relu'))
    
    model.add(layers.Dense(200, activation='relu'))
        
    model.add(layers.Dense(250, activation='relu'))
    
    model.add(layers.Dense(7, activation='linear'))
    
    
    model.compile(loss="mean_squared_error", optimizer="adam", metrics=[det_coeff])
             
    
    return model

model = initialize_model()

es = EarlyStopping(patience=25, restore_best_weights=True)
history = model.fit(X_train_scaled, y_train_scaled,
                    validation_split=0.3,
                    #validation_data=(X_test_scaled, y_test),
                    epochs=500, #play with this until your validation loss overfit
                    batch_size=32, # Keep batch size to 16 today
                    verbose=0,
                    callbacks=[es])

In [42]:
model.summary();

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_60 (Dense)             (None, 1500)              54000     
_________________________________________________________________
dense_61 (Dense)             (None, 250)               375250    
_________________________________________________________________
dense_62 (Dense)             (None, 200)               50200     
_________________________________________________________________
dense_63 (Dense)             (None, 250)               50250     
_________________________________________________________________
dense_64 (Dense)             (None, 7)                 1757      
Total params: 531,457
Trainable params: 531,457
Non-trainable params: 0
_________________________________________________________________


In [44]:
results = model.evaluate(X_test_scaled, y_test_scaled, verbose=0)
print(f'The r_square on the test set is of {results[1]:.2f}')

The r_square on the test set is of 0.81


In [45]:
results

[0.18985407054424286, 0.806775689125061]

In [46]:
y_pred= model.predict(X_test_scaled)

In [ ]:
data_df = pd.DataFrame(y_pred)

In [ ]:
data_df

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def plot_loss_accuracy(history, title=None):
    fig, ax = plt.subplots(1,2, figsize=(20,7))
    
    # --- LOSS --- 
    
    ax[0].plot(history.history['loss'])
    ax[0].plot(history.history['val_loss'])
    ax[0].set_title('Model loss')
    ax[0].set_ylabel('Loss')
    ax[0].set_xlabel('Epoch')
    ax[0].set_ylim((0,.5))
    ax[0].legend(['Train', 'Test'], loc='best')
    ax[0].grid(axis="x",linewidth=0.5)
    ax[0].grid(axis="y",linewidth=0.5)
    
    # --- ACCURACY
    
    ax[1].plot(history.history['mae'])
    ax[1].plot(history.history['val_mae'])
    ax[1].set_title('Model mean_absolute_error')
    ax[1].set_ylabel('mean_absolute_error')
    ax[1].set_xlabel('Epoch')
    ax[1].legend(['Train', 'Test'], loc='best')
    ax[1].set_ylim((0,0.5))
    ax[1].grid(axis="x",linewidth=0.5)
    ax[1].grid(axis="y",linewidth=0.5)
    
    if title:
        fig.suptitle(title)

In [ ]:
plot_loss_accuracy(history)